In [ ]:
# ******************************************
# yanrpi @2021
# ******************************************

import matplotlib.pyplot as plt

import numpy as np

from skimage import io
from skimage.feature import greycomatrix, greycoprops

# If you don't have sklearn:
# conda install scikit-learn

from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

import os
from os import path

In [ ]:
label_train = np.loadtxt('./ct_us/train.txt', dtype='int')
print(label_train.shape)
print(label_train)
number_of_training = label_train.shape[0]

folder_train = './ct_us/train'

filenames = [path.join(folder_train, 'train_{}.png'.format(i+1)) for i in range(number_of_training)]
#print(filenames)


In [ ]:
# randomly choose an image to demo
filename = filenames[8]
print(filename)

image = io.imread(filename)

plt.imshow(image, cmap='gray')
print(image.min(), image.max())

# 1.a GLCM
# https://scikit-image.org/docs/dev/api/skimage.feature.html?highlight=greycomatrix#skimage.feature.greycomatrix
comat = greycomatrix(image, [1,2,3], [0, np.pi/4, np.pi/2, 3*np.pi/4, np.pi], normed=True, symmetric=True)
comat[0:5,0:5,:,:] = 0 # Suppress background pixels
#comat /= comat.max()
print(comat.shape)

#print(comat[:,:,0,0])

In [ ]:
plt.figure()
for i in range(3):
    for j in range(5):
        plt.subplot(3, 5, i*5 + j + 1)
        plt.imshow(comat[:,:,i,j])
        plt.axis('off')
plt.tight_layout()
# 1.b Compute distances between two matrices (comat)

In [ ]:
# https://scikit-image.org/docs/dev/api/skimage.feature.html?highlight=greycoprops#skimage.feature.greycoprops
contrast = greycoprops(comat, 'contrast')
energy = greycoprops(comat, 'energy')

print(contrast.shape)
print(contrast)
print(energy)

In [ ]:
number_of_files = len(filenames)

distances = [1]
#distances = [1,2,3,4,5,6,7]
angles = [0, np.pi/4]
#angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
feature_all = np.zeros((number_of_files, len(distances) * len(angles)))

# Go through all the training images
for i in range(number_of_files):
    #print('Analyzing image {}'.format(filenames[i]))
    image = io.imread(filenames[i])
    
    #comat = greycomatrix(image, , normed=True, symmetric=True)
    comat = greycomatrix(image, distances, angles, normed=True, symmetric=True)
    comat[0:5,0:5] = 0 # Remove background influence
    
    # 2. Include other measures
    # * energy
    # * contrast
    # * dissimilarity
    # * homogeneity
    # * ASM
    # * correlation
    feature = greycoprops(comat, 'correlation')
    
    feature_all[i] = feature.ravel()


In [ ]:
#colors = ['red' if label_train[i] else 'blue' for i in range(number_of_files)]
#plt.scatter(feature_all[:,0], feature_all[:,1], c = colors, label = label_train)

In [ ]:
#X = np.concatenate((contrast_all, energy_all), axis=1)
#X = energy_all

# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
kmeans = KMeans(n_clusters=2, random_state=0).fit(feature_all)

print(kmeans.labels_)
print(label_train)

print('The classification accuracy is: {}'.format(accuracy_score(label_train, kmeans.labels_)))
print('The classification accuracy is: {}'.format(accuracy_score(label_train, 1 - kmeans.labels_)))